## 3.4 映射的弹性键查询

有时候为了方便起见，就算某个键在映射里不存在，我们也希望在通过这个键读取值的时候能得到一个默认值。  
有两个途径能帮我们达到这个目的:  
+ 一个是通过defaultdict 这个类型而不是普通的 dict
+ 另一个是给自己定义一个 dict 的子类，然后在子类中实现 \_\_missing\_\_ 方法。

### 3.4.1 defaultdict：处理找不到的键的一个选择

#### 示例 3-5 利用 defaultdict 实例而不是 setdefault 方法

In [1]:
"""创建一个从单词到其出现情况的映射"""
import sys
import re
import collections

WORD_RE = re.compile(r'\w+')

index = collections.defaultdict(list)  # 把 list 构造方法作为 default_factory 来创建一个 defaultdict

with open("file_for_exp3_2.txt", encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start()+1
            location = (line_no, column_no)
            index[word].append(location)

            
# 以字母顺序打印出结果
for word in sorted(index, key=str.upper):
    print(word, index[word])

aaa [(1, 1), (1, 13)]
bbb [(1, 5)]
ccc [(1, 9)]
ddd [(2, 1)]
eee [(2, 5)]
fff [(2, 9)]
hello [(3, 1)]
hey [(3, 10)]
hi [(3, 7)]


我们新建了这样一个字典：dd = defaultdict(list)，如果键 'new-key'在 dd 中还不存在的话，表达式 dd['new-key'] 会按照以下的步骤来行事。  
1. 调用 list() 来建立一个新列表。  
2. 把这个新列表作为值，'new-key' 作为它的键，放到 dd 中。  
3. 返回这个列表的引用。  

而这个用来生成默认值的可调用对象存放在名为 default_factory 的实例属性里。  

### 3.4.2 特殊方法\_\_missing\_\_  
#### 示例 3-7 StrKeyDict0 在查询的时候把非字符串的键转换为字符串

In [3]:
class StrKeyDict0(dict):  # StrKeyDict0 继承了 dict
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)  #  如果找不到的键本身就是字符串，那就抛出 KeyError 异常
        return self[str(key)]  # 如果找不到的键不是字符串，那么把它转换成字符串再进行查找

    def get(self, key, default=None):
        try:
            return self[key]  
            # get 方法把查找工作用 self[key] 的形式委托给 __getitem__，
            # 这样在宣布查找失败之前，还能通过 __missing__ 再给某个键一个机会。
        except KeyError:
            return default

    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()
        # 先按照传入键的原本的值来查找（我们的映射类型中可能含有非字符串的键），
        # 如果没找到，再用 str() 方法把键转换成字符串再查找一次

#### 示例 3-6 当有非字符串的键被查找的时候，StrKeyDict0 是如何在该键不存在的情况下，把它转换为字符串的 

In [4]:
d = StrKeyDict0([('2', 'two'), ('4', 'four')])

d['2']

'two'

In [5]:
d[4]  # 进行转换

'four'

In [7]:
d[1] # 会报 KeyError: '1'，因为首先将 1 转化为字符串 '1'，再进行查询，但是字典中不存在这个key值，所以报错

KeyError: '1'

In [8]:
d.get('2')

'two'

In [9]:
d.get(4)

'four'

In [10]:
d.get(1, 'N/A')  # default = 'N/A'

'N/A'

In [13]:
4 in d  # 调用__contains__函数

True

In [12]:
1 in d

False